# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [17]:
# Test API Key
geoapify_key

'b1e5ad87615943c6b740d31d926a3164'

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Transform Latitude and Longitude to float for linear regression
city_data_df["Lat"] = city_data_df["Lat"].astype(float)
city_data_df["Lng"] = city_data_df["Lng"].astype(float)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,poyarkovo,49.6000,128.6833,261.79,95,100,2.19,RU,1740430876
1,1,bilibino,68.0546,166.4372,232.90,98,40,1.41,RU,1740430878
2,2,kavarna,43.4333,28.3333,272.36,75,3,3.13,BG,1740430880
3,3,kone,-21.0595,164.8658,299.79,75,78,1.51,NC,1740430883
4,4,ribeira grande,38.5167,-28.7000,287.37,77,20,2.57,PT,1740430884


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Configure the map plot
city_data_df_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
)

# Display the map
city_data_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = city_data_df["Max Temp"] > 24
wind_speed = city_data_df["Wind Speed"] < 8

In [21]:
# Check to ensure there are values associated with criteria above
print(sum(max_temp))
print(sum(wind_speed))

578
527


In [22]:
# Display sample data
city_criteria_df = city_data_df[max_temp & wind_speed]
city_criteria_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,poyarkovo,49.6000,128.6833,261.79,95,100,2.19,RU,1740430876
1,1,bilibino,68.0546,166.4372,232.90,98,40,1.41,RU,1740430878
2,2,kavarna,43.4333,28.3333,272.36,75,3,3.13,BG,1740430880
3,3,kone,-21.0595,164.8658,299.79,75,78,1.51,NC,1740430883
4,4,ribeira grande,38.5167,-28.7000,287.37,77,20,2.57,PT,1740430884
...,...,...,...,...,...,...,...,...,...,...
572,572,callao,-12.0667,-77.1500,300.31,69,0,4.12,PE,1740431613
573,573,livny,52.4253,37.6069,262.14,96,48,3.09,RU,1740431614
575,575,togur,58.3631,82.8265,260.54,94,84,3.38,RU,1740431617
576,576,torghay,49.6339,63.4981,261.66,96,99,2.74,KZ,1740431619


In [23]:
# Drop any rows with null values
cleaned_city_criteria_df = city_criteria_df.dropna()
cleaned_city_criteria_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,poyarkovo,49.6000,128.6833,261.79,95,100,2.19,RU,1740430876
1,1,bilibino,68.0546,166.4372,232.90,98,40,1.41,RU,1740430878
2,2,kavarna,43.4333,28.3333,272.36,75,3,3.13,BG,1740430880
3,3,kone,-21.0595,164.8658,299.79,75,78,1.51,NC,1740430883
4,4,ribeira grande,38.5167,-28.7000,287.37,77,20,2.57,PT,1740430884
...,...,...,...,...,...,...,...,...,...,...
572,572,callao,-12.0667,-77.1500,300.31,69,0,4.12,PE,1740431613
573,573,livny,52.4253,37.6069,262.14,96,48,3.09,RU,1740431614
575,575,togur,58.3631,82.8265,260.54,94,84,3.38,RU,1740431617
576,576,torghay,49.6339,63.4981,261.66,96,99,2.74,KZ,1740431619


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_city_criteria_df.copy()

In [25]:
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,poyarkovo,49.6000,128.6833,261.79,95,100,2.19,RU,1740430876,
1,1,bilibino,68.0546,166.4372,232.90,98,40,1.41,RU,1740430878,
2,2,kavarna,43.4333,28.3333,272.36,75,3,3.13,BG,1740430880,
3,3,kone,-21.0595,164.8658,299.79,75,78,1.51,NC,1740430883,
4,4,ribeira grande,38.5167,-28.7000,287.37,77,20,2.57,PT,1740430884,
...,...,...,...,...,...,...,...,...,...,...,...
572,572,callao,-12.0667,-77.1500,300.31,69,0,4.12,PE,1740431613,
573,573,livny,52.4253,37.6069,262.14,96,48,3.09,RU,1740431614,
575,575,togur,58.3631,82.8265,260.54,94,84,3.38,RU,1740431617,
576,576,torghay,49.6339,63.4981,261.66,96,99,2.74,KZ,1740431619,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel" # As defined in the API documentation (key) for Geoapify

params = {
    "categories": categories,
    "limit": 5,
    "apiKey": geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)
    #name_address = requests.get(base_url)
    print(name_address.url)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]

    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=5&apiKey=b1e5ad87615943c6b740d31d926a3164&filter=circle%3A128.6833%2C49.6%2C10000&bias=proximity%3A128.6833%2C49.6
poyarkovo - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=5&apiKey=b1e5ad87615943c6b740d31d926a3164&filter=circle%3A166.4372%2C68.0546%2C10000&bias=proximity%3A166.4372%2C68.0546
bilibino - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=5&apiKey=b1e5ad87615943c6b740d31d926a3164&filter=circle%3A28.3333%2C43.4333%2C10000&bias=proximity%3A28.3333%2C43.4333
kavarna - nearest hotel: Добротица
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=5&apiKey=b1e5ad87615943c6b740d31d926a3164&filter=circle%3A164.8658%2C-21.0595%2C10000&bias=proximity%3A164.8658%2C-21.0595
kone - nearest hotel: Pacifik Appartels
https://api.geoapify.com/v2/places?categories=accommod

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,poyarkovo,49.6000,128.6833,261.79,95,100,2.19,RU,1740430876,No hotel found
1,1,bilibino,68.0546,166.4372,232.90,98,40,1.41,RU,1740430878,No hotel found
2,2,kavarna,43.4333,28.3333,272.36,75,3,3.13,BG,1740430880,Добротица
3,3,kone,-21.0595,164.8658,299.79,75,78,1.51,NC,1740430883,Pacifik Appartels
4,4,ribeira grande,38.5167,-28.7000,287.37,77,20,2.57,PT,1740430884,Hospedaria JSF
...,...,...,...,...,...,...,...,...,...,...,...
572,572,callao,-12.0667,-77.1500,300.31,69,0,4.12,PE,1740431613,Casa Ronald
573,573,livny,52.4253,37.6069,262.14,96,48,3.09,RU,1740431614,гостиница Ливны
575,575,togur,58.3631,82.8265,260.54,94,84,3.38,RU,1740431617,Визит
576,576,torghay,49.6339,63.4981,261.66,96,99,2.74,KZ,1740431619,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    tiles = "CartoLight",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    geo=True,
    color="City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)